## High-z galaxies with Te measurements

# Determine metallicities using the direct method

In [1]:
## Global stuff
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np
from astropy.table import Table


## Lines to be used
line_names = ['OIII1661','OIII1666','NIII1750','CIII1907','CIII1909','NeIII3869','OII3727','OII3729','OIITot',
              'Hd','Hg','OIII4363','Hb','OIII4959','OIII5007','Ha','NII6584']

#Probably there is a smart way of doing this with PyNeb
super_wav = {
    'OIII1661': 1661,
    'OIII1666': 1666,
    'NIII1750': 1750,
    'CIII1907': 1907,
    'CIII1909': 1909,
    'NeIII3869': 3869,
    'OII3727': 3727,
    'OII3729': 3729,
    'OIITot': 3729,
    'Hd': 4102,
    'Hg': 4340,
    'OIII4363': 4363,
    'Hb': 4861,
    'OIII4959': 4959,
    'OIII5007': 5007,
    'Ha': 6563,
    'NII6584': 6584
    }

# Quick look at all fluxes

In [2]:
# Load data
fluxes = Table.read('data_dered.dat',format='ascii.fixed_width_two_line')
fluxes.show_in_notebook()

## Using PyNeb/Izotov2006

In [3]:
## Temperatures from Izotov 2009
def Izotov06_TOIII(f,t):
    """ Equation 1 of Izotov 2006. Only works if OIII4663 is seen. Neglecting density.
    """
    t *= 1e-4
    CT = 8.44 - 1.09*t + 0.5*t**2 - 0.08*t**3 # Here we are neglecting the density
    new_t = 1.432 /(np.log10((f['OIII4959'] + f['OIII5007'])/f['OIII4363']) - np.log10(CT))
    return new_t * 1e4

def calculate_TOIII(f):
    
    params = Parameters()
    params.add('t', value=10000., min=1000., max=30000.)
    
    def fnc(params,f):
        t = params['t']
        return t - Izotov06_TOIII(f,t)
    
    fit = minimize(fnc, params,args=(f,),method='least_squares')
    
    return fit.params['t'].value
    
    
def Izotov06_TOII(tOIII,Z='intermediate'):
    """Equation 14 of Izotov 2006. The dispersion of this relation is not provided
    """
    tOIII *= 1e-4 
    if Z == 'low': # 7.2
        return (-0.577 + tOIII * (2.065 - 0.498*tOIII))*1e4 
    elif Z == 'intermediate': #7.6
         return (-0.744 + tOIII * (2.338 - 0.610*tOIII))*1e4 
    elif Z == 'high': # 8.2
        return (2.967 + tOIII * (-4.797 + 2.827*tOIII))*1e4 
    else:
        print('Wrong "Z" option')

In [4]:
# Calculate metallicities
import pyneb as pn
pn.atomicData.setDataFile('o_iii_coll_AK99.dat') # have to change because the default ones do not have level 6
O3 = pn.Atom('O', 3)
O2 = pn.Atom('O',2)

dered_fluxes = Table.read('data_dered.dat',format='ascii.fixed_width_two_line')
Te_data = Table.read('data_Te.dat',format='ascii.fixed_width_two_line')
Te_data.add_index('Name')

O_H_pyneb = []
O2_H_pyneb = []
met_pyneb = []

def sum_met(OH,O2H):
    return 12 + np.log10(10**(OH-12)+10**(O2H-12))

for f in dered_fluxes:
    
    Te = Te_data.loc[f['Name']]['Te']
    Den = Te_data.loc[f['Name']]['Den']
    
    #  Calculate TOII. take our estimated temperatures as TOIII.
    t_low = (Izotov06_TOII(Te,Z='low')) # May need to change the Z
    t_inter = (Izotov06_TOII(Te,Z='intermediate')) # May need to change the Z
    t_high = (Izotov06_TOII(Te,Z='high')) # May need to change the Z

    def change_scale(x):
        return 12+np.log10(x)
    
    # Calculate metalliticites
    O2_H_pyneb.append(change_scale(O3.getIonAbundance((f['OIII4959']+f['OIII5007']),Te,Den,
                                                to_eval='L(4959)+L(5007)',Hbeta=f['Hb'])))
    
    O_H_pyneb.append(change_scale(O2.getIonAbundance((f['OIITot']),t_inter,Den,
                                                     to_eval='L(3726)+L(3729)',Hbeta=f['Hb'])))
    
    met_pyneb.append(sum_met(O2_H_pyneb[-1],O_H_pyneb[-1]))
    
    a = change_scale(O2.getIonAbundance((f['OIITot']),t_low,Den, to_eval='L(3726)+L(3729)',Hbeta=f['Hb']))
    b = change_scale(O2.getIonAbundance((f['OIITot']),t_inter,Den, to_eval='L(3726)+L(3729)',Hbeta=f['Hb']))
    c = change_scale(O2.getIonAbundance((f['OIITot']),t_high,Den, to_eval='L(3726)+L(3729)',Hbeta=f['Hb']))
    #print(a,b,c)
    
    
print('                Name\tO/H\tO+2/H\tMetOH')
for i,f in enumerate(dered_fluxes):
    print('%20s\t%0.2f\t%0.2f\t%0.2f'%(f['Name'],O_H_pyneb[i],O2_H_pyneb[i],met_pyneb[i]))


                Name	O/H	O+2/H	MetOH
              CSWA20	7.01	7.96	8.01
       Abell_860_359	7.28	8.01	8.08
          Abell_22.3	7.27	7.36	7.62
               RCSGA	7.75	7.98	8.18
          A1689_31.1	6.75	7.41	7.50
          SMACS_0304	7.85	8.02	8.25
           MACS_0451	6.73	7.34	7.44
        COSMOS_12805	7.70	8.07	8.22
               BX660	6.92	7.84	7.89
                BX74	7.15	7.97	8.03
               BX418	7.02	7.90	7.95
           S16-stack	7.75	7.94	8.15
         COSMOS-1908	6.80	7.90	7.94
        the_Lynx_arc	6.85	7.79	7.83
          SMACS_2031	6.84	7.62	7.68
           SGAS_1050	7.41	8.36	8.41


## Calculate Errors

In [5]:
def change_scale(x):
    return 12+np.log10(x)


def perturbed_lines(f,nb):
    np.random.seed()
    new_lines = {}
    for line in ['OIII4959',"OIII5007","OIITot",'Hb']:
        if np.isfinite(f[line]) and np.isfinite(f['e'+line]):
            new_lines[line] = np.random.normal(f[line],f['e'+line],nb)
        else:
            new_lines[line] = [f[line]] * nb

    return new_lines


In [26]:
# Calculate metallicities with error estimation
from astropy.visualization import hist
import pickle
galaxies = Table.read('data_dered.dat',format='ascii.fixed_width_two_line')

O_H_pyneb = []
O2_H_pyneb = []
met_pyneb = []
upO_H_pyneb = []
upO2_H_pyneb = []
upmet_pyneb = []
lowO_H_pyneb = []
lowO2_H_pyneb = []
lowmet_pyneb = []

nb = 500

print('                Name\t       O/H\t\t       O+2/H\t\t      MetOH')
for gal in galaxies:
    
    O2_H_sample = []
    O_H_sample = []
    
    #  Sample of fluxes, densitties and TOIII. Calculate TOII
    dered_f = perturbed_lines(gal,nb)
    tem_and_den    = pickle.load( open( 'tem_and_ebv_samples/'+gal['Name']+"_tem_and_ebv.pickle", "rb" ) )
    tOIII_sample  = np.random.choice(tem_and_den['tem'], nb)#np.full_like(dered_f['Hb'],gal['Te'])
    den_sample    = np.random.choice(tem_and_den['den'], nb)#np.full_like(dered_f['Hb'],gal['Den'])  
    tOII_sample   = np.array([Izotov06_TOII(x,Z='low') for x in tOIII_sample]) # May need to change the Z
    
    ## Calculate metalliticites
    O2_H_sample.append(O3.getIonAbundance(np.array(dered_f['OIII4959'])+np.array(dered_f['OIII5007']),
                                          tOIII_sample,den_sample,to_eval='L(4959)+L(5007)',Hbeta=dered_f['Hb']))
    O_H_sample.append(O2.getIonAbundance(np.array(dered_f['OIITot']),
                                          tOII_sample,den_sample,
                                          to_eval='L(3726)+L(3729)',Hbeta=dered_f['Hb']))
    
    total = np.array([change_scale(m1+m2) for  m1,m2 in zip(O2_H_sample,O_H_sample)])
    O2_H_sample = np.array([change_scale(m) for  m in O2_H_sample])
    O_H_sample  = np.array([change_scale(m) for  m in O_H_sample])
    
    O2_H_pyneb.append(np.nanpercentile(O2_H_sample,50))
    O_H_pyneb.append(np.nanpercentile(O_H_sample,50)) 
    upO2_H_pyneb.append(np.nanpercentile(O2_H_sample,84)-np.nanpercentile(O2_H_sample,50))
    upO_H_pyneb.append(np.nanpercentile(O_H_sample,84)-np.nanpercentile(O_H_sample,50))
    lowO2_H_pyneb.append(np.nanpercentile(O2_H_sample,50)-np.nanpercentile(O2_H_sample,16))
    lowO_H_pyneb.append(np.nanpercentile(O_H_sample,50)-np.nanpercentile(O_H_sample,16))
    met_pyneb.append(np.nanpercentile(total,50))
    upmet_pyneb.append(np.nanpercentile(total,84)-np.nanpercentile(total,50))
    lowmet_pyneb.append(np.nanpercentile(total,50)-np.nanpercentile(total,16))

    print('%20s\t%0.2f + %0.2f - %0.2f\t%0.2f + %0.2f - %0.2f\t%0.2f + %0.2f - %0.2f'%(gal['Name'],
                                                               O_H_pyneb[-1],upO_H_pyneb[-1],lowO_H_pyneb[-1],
                                                               O2_H_pyneb[-1],upO2_H_pyneb[-1],lowO2_H_pyneb[-1],
                                                              met_pyneb[-1],upmet_pyneb[-1],lowmet_pyneb[-1]))


                Name	       O/H		       O+2/H		      MetOH
              CSWA20	6.99 + 0.43 - 0.31	7.96 + 0.33 - 0.21	8.01 + 0.34 - 0.21
       Abell_860_359	7.29 + 0.14 - 0.09	8.02 + 0.13 - 0.10	8.09 + 0.13 - 0.10
          Abell_22.3	7.06 + 0.30 - 0.16	7.40 + 0.14 - 0.11	7.57 + 0.09 - 0.08
               RCSGA	7.75 + 0.03 - 0.04	7.98 + 0.02 - 0.03	8.18 + 0.02 - 0.03
          A1689_31.1	6.72 + 0.11 - 0.03	7.42 + 0.17 - 0.15	7.50 + 0.16 - 0.10
          SMACS_0304	7.87 + 0.29 - 0.12	8.03 + 0.06 - 0.07	8.27 + 0.15 - 0.09
           MACS_0451	6.70 + 0.07 - 0.05	7.35 + 0.16 - 0.10	7.44 + 0.14 - 0.09
        COSMOS_12805	7.68 + 0.18 - 0.18	8.05 + 0.17 - 0.22	8.21 + 0.18 - 0.21
               BX660	6.92 + 0.13 - 0.10	7.83 + 0.14 - 0.17	7.88 + 0.13 - 0.17
                BX74	7.14 + 0.16 - 0.16	7.96 + 0.18 - 0.30	8.02 + 0.17 - 0.27
               BX418	7.03 + 0.16 - 0.15	7.90 + 0.15 - 0.18	7.96 + 0.15 - 0.18
           S16-stack	7.75 + 0.07 - 0.07	7.94 + 0.06 - 0.06	8.15 + 0.06 - 0.06
     

In [27]:
## Save results in a table 
pyneb_out = Table([galaxies['Name'],
                   O_H_pyneb,upO_H_pyneb,lowO_H_pyneb,
                   O2_H_pyneb,upO2_H_pyneb,lowO2_H_pyneb,
                   met_pyneb,upmet_pyneb,lowmet_pyneb],
                names = ['Name','O/H','upO/H','lowO/H','O2/H','upO2/H','lowO2/H','met','upmet','lowmet'])

pyneb_out.write('direct_metallicity.dat',format='ascii.fixed_width_two_line')
pyneb_out.show_in_notebook() 

In [44]:
## Write it to paper 
sample = Table.read('../Data/galaxy_properties.dat',format='ascii.fixed_width_two_line') # just to get the right order
Te = Table.read('data_Te.dat',format='ascii.fixed_width_two_line')
Te.add_index('Name')
Met = Table.read('direct_metallicity.dat',format='ascii.fixed_width_two_line')
Met.add_index('Name')

diag = ['b,c','a,c','a','b','b,c','a,c','b','a','a','a','a','a,c','b,c','a,d','a,c','a,c']
    
out = open('TeMet_for_paper','w')
out.write('Object & Diag. & \Te & \Ne & \ext & 12+log(O$^+$/H$^+$) & 12+log(O$^{2+}$/H$^+$) & 12+log(O/H)$_{(Te)}$ \n')
          
for gal,d in zip(sample,diag):
    m = Te.loc[gal['Name']]
    out.write('%s\t&%s\t&'%(m['Name'],d))
    out.write('%d$^{+%d}_{-%d}$\t&'%(m['Te'],m['upTe']-m['Te'],m['Te']-m['lowTe']))
    out.write('%d$^{+%d}_{-%d}$\t&'%(m['Den'],m['upDen']-m['Den'],m['Den']-m['lowDen']))
    out.write('%0.2f$^{+%0.2f}_{-%0.2f}$\t &'%(m['EBV'],m['upEBV']-m['EBV'],m['EBV']-m['lowEBV']))

    t = Met.loc[gal['Name']]
    out.write('%0.2f$^{+%0.2f}_{-%0.2f}$\t &'%(t['O/H'],t['upO/H'],t['lowO/H']))  
    out.write('%0.2f$^{+%0.2f}_{-%0.2f}$\t &'%(t['O2/H'],t['upO2/H'],t['lowO2/H']))   
    out.write('%0.2f$^{+%0.2f}_{-%0.2f}$\t '%(t['met'],t['upmet'],t['lowmet']))  
    out.write('\\\\ \n')
                    
out.close()

In [32]:
# Compare with previous estimations
plt.rcParams.update({'font.size': 12})
galaxies = Table.read('../Data/galaxy_properties.dat',format='ascii.fixed_width_two_line')
Te = Table.read('direct_metallicity.dat',format='ascii.fixed_width_two_line')
Te.add_index('Name')

## colors and markers
colors = plt.get_cmap('cubehelix')(np.linspace(0.1, 0.8, len(galaxies['Name'])))

#plotting
fig, ax = plt.subplots(1,1,figsize=(5,5))
fig.subplots_adjust(left=0.2)
diff = []

for gal in galaxies: 
    met = Te.loc[gal['Name']]
    diff.append(np.abs(gal['MetTe']-met['met']))
    ax.errorbar(gal['MetTe'],met['met'],label=gal['Name'],
                xerr=np.array((np.abs(gal['infMetTe']),gal['supMetTe'])).reshape(2,1),
                yerr=np.array((met['lowmet'],met['upmet'])).reshape(2,1),
                color=colors[gal['nb']],marker=gal['marker'],
                markersize=7,elinewidth=0.8)

ax.plot(np.arange(6.0,10.),np.arange(6.0,10.),color='k',linewidth=2.0,linestyle=':')
ax.plot(np.arange(6.0,10.),np.arange(6.0,10.)+0.5,color='k',linewidth=0.7,linestyle=':')
ax.plot(np.arange(6.0,10.),np.arange(6.0,10.)-0.5,color='k',linewidth=0.7,linestyle=':')
ax.set_ylabel('12+log(O/H) $_{This\,\,\,Work}$')
ax.set_xlabel('12+log(O/H) $_{Literature}$')
ax.set_ylim(7.0,9.0)
ax.set_xlim(7.0,9.0)

print(np.mean(diff),np.std(diff))

fig.savefig('/Users/vera/Desktop/DirectMethod_Lit.pdf')

<IPython.core.display.Javascript object>

(0.10596576972375005, 0.073478306638522672)
